Com problemas quando tem ruído, usar stream mux??

In [1]:
from scipy.io import wavfile
fs, data = wavfile.read("song.wav")

In [2]:
with open("song.wav", "rb")as f:
    _ = f.read()

In [3]:
data.shape

(4454100, 2)

In [4]:
data.dtype

dtype('int16')

In [5]:
#data2 = data[:4454100//8].ravel()

In [6]:
eightBit = np.array(((data[:len(data)//8] + 32768)) >> 8, dtype = np.uint8)

In [7]:
wavfile.write("reduced.wav", fs , eightBit)

In [8]:
prefix = np.random.randint(0, 256, 100, dtype = np.uint8)
prefix

array([171,   6,  59, 217,  74,  70, 120, 176,   9, 238, 214,  40, 157,
       244, 103, 118, 136,  58,  80, 176,  43, 143, 228, 184,  73, 206,
       130, 201,  36, 103,  49,  95,  54, 145, 185, 242, 170, 246, 134,
       234,  41,  75, 246,  54, 144, 138, 230,  65, 158,  43, 251,   8,
       109,  46, 226, 204,  52, 195, 152, 205,  31, 201, 232, 136,  79,
         0, 217,  90, 243, 105,  56,  85,  59, 248, 181, 200, 155, 233,
       135,  28, 225,  86, 213, 150, 130, 232,   4,  43, 130, 239,   6,
       186, 254,  57,  18,  92,  41, 115, 210,  50], dtype=uint8)

In [9]:
with open("prefixData.data", "wb") as f:
    f.write(prefix)
    f.write(eightBit)

In [10]:
with open("output.data", "rb") as f:
    _ = f.read()

rcv = np.frombuffer(_, np.uint8)

In [11]:
rcv

array([248,   7, 248, ...,  31, 224,  31], dtype=uint8)

In [12]:
def find_subsequence(seq, subseq):
    target = np.dot(subseq, subseq)
    candidates = np.where(np.correlate(seq,subseq, mode='valid') == target)[0]
    # some of the candidates entries may be false positives, double check
    check = candidates[:, np.newaxis] + np.arange(len(subseq))
    mask = np.all((np.take(seq, check) == subseq), axis=-1)
    return candidates[mask]

In [13]:
find_subsequence(np.unpackbits(rcv), np.unpackbits(prefix)[:18])

array([8017378])

In [14]:
rcv2 = np.roll(np.packbits(np.roll(np.unpackbits(rcv), -8017378 )), 0)

In [15]:
final_data = rcv2[len(prefix):].reshape(-1, 2)

In [53]:
np.mean(np.equal(eightBit.ravel(), final_data.ravel()))

0.5660910766180163

In [33]:
eightBit.ravel()[-100:]

array([103, 111, 101, 113, 101, 115, 100, 116,  98, 118,  97, 119,  96,
       120,  96, 121,  95, 121,  95, 121,  96, 121,  97, 121,  98, 120,
        98, 120,  99, 120,  99, 120,  99, 119,  99, 119,  99, 118, 100,
       118, 101, 118, 102, 118, 103, 118, 104, 118, 105, 119, 105, 121,
       104, 122, 104, 124, 103, 126, 102, 128, 102, 130, 101, 131, 101,
       132, 100, 132, 100, 132, 100, 132, 100, 132, 101, 132, 102, 132,
       103, 132, 105, 132, 106, 132, 107, 133, 108, 134, 109, 134, 109,
       135, 109, 136, 110, 136, 110, 137, 111, 138], dtype=uint8)

In [35]:
final_data.ravel()[-100:]

array([103, 111, 101, 113, 101, 115, 100, 116,  98, 118,  97, 119,  96,
       120,  96, 121,  95, 121,  95, 121,  96, 121,  97, 121,  98, 120,
        98, 120,  99, 120,  99, 120,  99, 119,  99, 119,  99, 118, 100,
       118, 101, 118, 102, 118, 103, 118, 104, 118, 105, 119, 105, 121,
       104, 122, 104, 124, 103, 126, 102, 128, 102, 130, 101, 131, 101,
       132, 100, 132, 100, 132, 100, 132, 100, 132, 101, 132, 102, 132,
       103, 132, 105, 132, 106, 132, 107, 133, 108, 134, 109, 134, 109,
       135, 109, 136, 110, 136, 110, 137, 111, 138], dtype=uint8)

In [19]:
wavfile.write("final.wav", fs, final_data)